In [3]:
import pickle
from enum import Enum
from typing import Any, Dict, List
from pydantic import BaseModel, validator

from langchain.document_loaders import ReadTheDocsLoader
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.vectorstores.base import VectorStore
from langchain.callbacks.base import AsyncCallbackHandler
from langchain.callbacks.manager import AsyncCallbackManager
from langchain.callbacks.tracers import LangChainTracer
from langchain.chains import ChatVectorDBChain, ConversationalRetrievalChain
from langchain.chains.llm import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains.chat_vector_db.prompts import (CONDENSE_QUESTION_PROMPT,
                                                     QA_PROMPT)
from langchain.chat_models import ChatOpenAI


In [15]:

def ingest_document() -> VectorStore: 
    """Load documents from web pages
    This step allows to create embeddings from raw text, splitted by chunk. 
    The embeddings are stored in a vectorstore database and can be used for further processing.
    """
    loader = ReadTheDocsLoader("langchain.readthedocs.io/en/latest/")
    raw_documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    documents = text_splitter.split_documents(raw_documents)
    embeddings = OpenAIEmbeddings(openai_api_key="YOUR_API_KEY") # type: ignore
    vectorstore = FAISS.from_documents(documents, embeddings)

    #save vectorstore
    with open("vectorstore.pickle", "wb") as f:
        pickle.dump(vectorstore, f)
    
    return vectorstore


vectorstore = ingest_document()
llm = ChatOpenAI(model_name="gpt-3.5-turbo", verbose=True)  # type: ignore
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=False)
chat_qa = ConversationalRetrievalChain.from_llm(llm=llm, 
                                        retriever=vectorstore.as_retriever(), 
                                        condense_question_prompt=CONDENSE_QUESTION_PROMPT, 
                                        qa_prompt=QA_PROMPT, 
                                        #return_source_documents=True,
                                        memory=memory,
                                        get_chat_history = lambda x: x, 
                                        verbose=True)

/Users/thanhtu/Desktop/perso/langchain-examples/.venv/lib/python3.9/site-packages/langchain/document_loaders/readthedocs.py:48: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 48 of the file /Users/thanhtu/Desktop/perso/langchain-examples/.venv/lib/python3.9/site-packages/langchain/document_loaders/readthedocs.py. To get rid of this warning, pass the additional argument 'features="html5lib"' to the BeautifulSoup constructor.

  _ = BeautifulSoup(


IndexError: list index out of range

In [16]:
chat_qa.run(input="How you can help me?")

NameError: name 'chat_qa' is not defined

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    chatbot = gr.Chatbot([], elem_id="chatbot").style(height=500)
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]
    
    def bot(history):
        print(history)
        bot_message = chat_qa.run({"question": history[-1][0], "chat_history" : history[:-1]})
        history[-1][1] = bot_message
        return history
    
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(bot, chatbot, chatbot)
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()